#Decision Tree

each item has two continuous features
class label is {0, 1}



In [28]:
import numpy as np
import pandas as pd
import math

df = pd.read_csv('./data/D1.txt', sep='\s+', names=['x1', 'x2', 'y'])
df.columns[:-1]

Index(['x1', 'x2'], dtype='object')

In [25]:
def determine_candidate_numeric_splits(D: pd.DataFrame, X: str) -> (str, float):
    C = []
    instances = D.sort_values(by=X)
    instances = instances.reset_index(drop=True)
    for i, row in instances.iterrows():
        if i < len(instances) - 1:
            y_i = row['y']
            y_next = instances.at[i + 1, 'y']
            if y_i != y_next:
                x_i = row[X]
                x_next = instances.at[i + 1, X]
                C.append((X, x_i)) # x_i >= c
                # C.append((X, (x_i + x_next) / 2))
    return C

In [27]:
def determine_candidate_splits(D: pd.DataFrame) -> (str, float):
    C = []
    for feature in D.columns[:-1]:
        C.append(determine_candidate_numeric_splits(D, feature))
    return C

In [36]:
# GainRatio(D, Split) = (H(Y) - H(Y | Split)) / H(Split)

def entropy_Y(D: pd.DataFrame, Y: str) -> float:
    # H(Y) = -Sum(P(y)*log2(P(y)) for each y in Y)
    H = 0
    y = D[Y].unique()
    for y_i in y:
        Py_i = sum(D[Y] == y_i) / len(D[Y])
        H += Py_i * math.log2(Py_i)
    H *= -1
    return H

def entropy_S(D: pd.DataFrame, S: (str, float)) -> float:
    H = 0
    X, c = S
    Pgreater = sum(D[X] >= c) / len(D[X])
    H += Pgreater * math.log2(Pgreater)
    Plesser = sum(D[X] < c) / len(D[X])
    H += Plesser * math.log2(Plesser)
    H *= -1
    return H

def conditional_entropy(D: pd.DataFrame, Y: str, S: (str, float)) -> float:
    # H(Y | Split) = (P(X >= c) * H(Y | X >= c)) + (P(X < c) * H(Y | X < c))
    H = 0
    X, c = S
    HYgreater = entropy_Y(D[D[X] >= c], Y)
    Pgreater = sum(D[X] >= c) / len(D[X])
    HYlesser = entropy_Y(D[D[X] < c], Y)
    Plesser = sum(D[X] < c) / len(D[X])
    H += Pgreater * HYgreater
    H += Plesser * HYlesser
    return H

#print(entropy_Y(df, 'y'))
#print(entropy_S(df, ('x1', .5)))
#print(conditional_entropy(df, 'y', ('x2', .2)))

In [ ]:
def make_subtree(D: pd.DataFrame):
    stop = False
    best_split = None

    # if node is empty or all splits have 0 info or any split has 0 entropy
    if len(D) == 0:
        stop = True
    else:
        candidate_splits = determine_candidate_splits(D)
        gainRatios = pd.Series()
        for split in candidate_splits:
            split_entropy = entropy_S(D, split)
            if split_entropy == 0:
                stop = True ##
                break
            else:
                # GainRatio(D, Split) = (H(Y) - H(Y | Split)) / H(Split)
                gainRatio = (entropy_Y(D, 'y') - conditional_entropy(D, 'y', split)) / entropy_S(D, split)
                gainRatios.append(gainRatio)
        best_split = candidate_splits[gainRatios.idxmax()]

    if stop:
        # make leaf node with class label
    else:
        # make internal node with best split
        # create children
            
    # if stopping criteria:
    #   make a leaf node N
    #   determine class label of N
    # else:
    #   make internal node N
    #   S = FindBestSplit(D, candidate_splits)
    #   for each group k of S
    #       Dk := subset of training data in group k
    #       kth child of N := make_subtree(Dk)
    # return subtree rooted at N
    return

dtree = make_subtree(df[:])